<a href="https://colab.research.google.com/github/Mr22XX/shortest_path_djikstra/blob/main/shortest_path.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import openrouteservice
import folium
import heapq

# === 1. API KEY ORS ===
client = openrouteservice.Client(key='5b3ce3597851110001cf6248aaa63c5678d84f509bc13522adb804d3')

# === 2. Koordinat Lokasi Kampus ===
locations = {
    "Rektorat": [102.27231460986346, -3.7590495172423495],
    "Perpustakaan": [102.27485462111163, -3.756806076798016],
    "Danau Unib": [102.27306446726114, -3.758445851629117],
    "LPTIK": [102.27501541748192, -3.7585034389347047],
    "GSG": [102.27655797563433, -3.757536160753844],
    "GB 1": [102.27372095056845, -3.7568032921655625],
    "GB 2": [102.274037554275, -3.7578575751002457],
    "GB 3 & 4": [102.27664495427499, -3.7560850630710587],
    "Stadion Unib": [102.27817155070424, -3.7576442412116946],
    "Gedung FKIP": [102.27746551161644, -3.756364599659421],
    "GLT": [102.27191958168342, -3.75809920273443]
}

# === 3. Graf Koneksi Manual ===
graph = {
    "Rektorat": ["Danau Unib", "Perpustakaan", "GLT"],
    "Danau Unib": ["Rektorat", "LPTIK"],
    "LPTIK": ["Danau Unib", "GSG"],
    "Perpustakaan": ["Rektorat", "GB 1"],
    "GB 1": ["Perpustakaan", "GB 2"],
    "GB 2": ["GB 1", "GB 3 & 4"],
    "GB 3 & 4": ["GB 2", "Stadion Unib"],
    "Stadion Unib": ["GB 3 & 4", "Gedung FKIP"],
    "Gedung FKIP": ["Stadion Unib"],
    "GLT": ["Rektorat"]
}

# === 4. Hitung Bobot Jarak Jalan via ORS ===
weighted_graph = {}
for node in graph:
    weighted_graph[node] = {}
    for neighbor in graph[node]:
        coord_start = locations[node]
        coord_end = locations[neighbor]
        try:
            route = client.directions(
                coordinates=[coord_start, coord_end],
                profile='driving-car',
                format='geojson'
            )
            distance_m = route['features'][0]['properties']['summary']['distance']
            weighted_graph[node][neighbor] = distance_m
        except Exception as e:
            print(f"Gagal mendapatkan jarak dari {node} ke {neighbor}: {e}")

# === 5. Algoritma Dijkstra ===
def dijkstra(graph, start, end):
    queue = [(0, start, [])]
    visited = set()
    while queue:
        (cost, node, path) = heapq.heappop(queue)
        if node in visited:
            continue
        path = path + [node]
        if node == end:
            return (cost, path)
        visited.add(node)
        for neighbor, weight in graph.get(node, {}).items():
            if neighbor not in visited:
                heapq.heappush(queue, (cost + weight, neighbor, path))
    return (float("inf"), [])

# === 6. Input Lokasi ===
start_input = input("Masukkan Lokasi Awal: ").strip()
end_input = input("Masukkan Lokasi Akhir: ").strip()

# === 7. Jalankan Dijkstra ===
total_dist, dijkstra_path = dijkstra(weighted_graph, start_input, end_input)

# === 8. Inisialisasi Peta ===
m = folium.Map(location=[-3.759, 102.273], zoom_start=17)

# === 9. Tambahkan Marker Lokasi ===
for name, coord in locations.items():
    folium.Marker(
        location=[coord[1], coord[0]],
        popup=name,
        icon=folium.Icon(color='green' if name == start_input else 'blue' if name == end_input else 'gray')
    ).add_to(m)

# === 10. Tampilkan Jalur Dijkstra via Jalan ===
dijkstra_coords_full = []

for i in range(len(dijkstra_path) - 1):
    start = locations[dijkstra_path[i]]
    end = locations[dijkstra_path[i + 1]]
    try:
        route_segment = client.directions(
            coordinates=[start, end],
            profile='driving-car',
            format='geojson'
        )
        segment_coords = route_segment['features'][0]['geometry']['coordinates']
        dijkstra_coords_full.extend(segment_coords)
    except Exception as e:
        print(f"Gagal ambil rute {dijkstra_path[i]} -> {dijkstra_path[i+1]}: {e}")

# Tambahkan PolyLine ke Peta
folium.PolyLine(
    locations=[[latlng[1], latlng[0]] for latlng in dijkstra_coords_full],
    color='red', weight=6, opacity=0.8,
    tooltip='Rute Dijkstra (Jalur Jalan)'
).add_to(m)

# === 11. Tampilkan Rute Alternatif ORS ===
colors = ['blue', 'orange', 'purple', 'green']
start_coord = locations[start_input]
end_coord = locations[end_input]

routes = client.directions(
    coordinates=[start_coord, end_coord],
    profile='driving-car',
    format='geojson',
    alternative_routes={"share_factor": 0.6, "target_count": 3}
)

print("\n=== RUTE TERPENDEK DENGAN DIJKSTRA ===")
print(" -> ".join(dijkstra_path))
print(f"Total Estimasi Jarak: {total_dist/1000:.2f} km (via Jalan)\n")

print("=== PERBANDINGAN RUTE ALTERNATIF  ===")
for i, feature in enumerate(routes['features']):
    props = feature['properties']['summary']
    distance_km = props['distance'] / 1000
    duration_min = props['duration'] / 60

    folium.GeoJson(
        feature,
        name=f'Rute {i + 1}',
        tooltip=f'Rute {i + 1}: {distance_km:.2f} km, {duration_min:.2f} menit',
        style_function=lambda x, col=colors[i % len(colors)]: {
            'color': col, 'weight': 4, 'opacity': 0.7
        }
    ).add_to(m)

    print(f"Rute {i + 1}:")
    print(f"  - Jarak : {distance_km:.2f} km")
    print(f"  - Waktu : {duration_min:.2f} menit\n")

# === 12. Tampilkan Peta Langsung ===
m


/usr/local/lib/python3.11/dist-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/usr/local/lib/python3.11/dist-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/usr/local/lib/python3.11/dist-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 4th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/usr/local/lib/python3.11/dist-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 5th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,


Masukkan Lokasi Awal: Rektorat
Masukkan Lokasi Akhir: GSG

=== RUTE TERPENDEK DENGAN DIJKSTRA ===
Rektorat -> Danau Unib -> LPTIK -> GSG
Total Estimasi Jarak: 1.07 km (via Jalan)

=== PERBANDINGAN RUTE ALTERNATIF ORS ===
Rute 1:
  - Jarak : 1.08 km
  - Waktu : 3.79 menit

Rute 2:
  - Jarak : 1.11 km
  - Waktu : 4.26 menit

Rute 3:
  - Jarak : 1.22 km
  - Waktu : 4.36 menit

